# **HOMEWORK 4 - Topics in Databases - Edward Miller**



## Please add the following to the database you have created in HW3:


## Due December 1, 2023

The first thing to do is to establish a database, before adding any assertions or triggers to it. If it's already established, we can go to the next steps, but if it's not, we need to establish it.

In [ ]:
pip install psycopg2-binary

In [ ]:
pip install tabulate

In [ ]:
import psycopg2
from psycopg2 import sql
import tabulate
from tabulate import tabulate
from getpass4 import getpass
# importing libraries

In [ ]:
Admin_Pass = input("Enter the Admin Password\n")
# Getting the database password

First to make functions to check if a database exists and if it doesn't to create that database

In [ ]:
yes_choices = ["yes","y","ye", "yep", "ok", "okay", "sure"]
# sets up a list containing yes options

def check_if_database_exists(name):
    
    '''    
    This function checks to see if a database has already been established.
    If it does exist, it returns a value of True, and if not, it returns False
    and prints out a list of the databases, in order for the user to see which
    databases one might select if the user chooses not 
    
        Args: 
            name - a string denoting the database name
        Return: 
            check - a boolean value of True or False  
            
    '''
    
    conn = psycopg2.connect(
    #database=database_name, 
    user='postgres', 
    #password = Admin_Pass,
    password=Admin_Pass, 
    ##host='localhost', 
    host='127.0.0.1', 
    port= '5434'
    )
    conn.autocommit = True
    # open connection
    
    check = False
    # initializing variable
    cur = conn.cursor()
    # setting up cursor
    cur.execute("SELECT datname FROM pg_database;")
    # getting list of databases
    list_database = cur.fetchall()
    # gets everything 
    cur.close()
    if (name,) in list_database:
        print("'{}' database already exists in this location\n".format(name))
        check = True
    else:
        print("'{}' database does not exist in this location.\n".format(name))
        print("Here are the databases currently in this location")
        for database in list_database:             
            print(database)
    return check

def create_a_database(name):
    
    '''    
    This function creates a database in the local PostgreSQL location
    
        Args: 
            name - a string denoting the database name
        Return: 
            none
            
    '''       
    conn = psycopg2.connect(
        user='postgres', 
        password=Admin_Pass, 
        host='127.0.0.1', 
        port= '5434'
    )
    conn.autocommit = True
    # open connection
    
    cursor = conn.cursor()
    # Creating a cursor object

    sql_command = sql.SQL("CREATE database {database}").format(database=sql.Identifier(name))
    # establishing sql query to create a database 
 
    cursor.execute(sql_command)
    # executing above query
    print("Database has been created successfully !!");
    # confirmation message
    cursor.close()
    # close the cursor
    conn.close()    
    # Closing the connection
    return

After creating the necessary functions to check whether a database exists, and also create one if needed, the next step is to add tables for the database

In [ ]:
database_name = 'Assignment 4'
# arbitrary database for this assignment

if check_if_database_exists(database_name) == False:

    create_yes = input("Would you like to create a database using this name? ") 
    if create_yes.lower() in yes_choices: 
        # looks in list to see if user selected some form of "yes"
        # lower() used to disregard capital letters
        
        print("Creating database - %s" %database_name)
        create_a_database(database_name)
        

Now to check the tables in the database, and if the needed tables are not in the database, make a function to create those tables

In [ ]:
def print_table_list(database):  
    
    '''    
    This function gets a list of tables 
    that are in the database 
    
        Args: 
            database - a string referencing the database containing the tables
        Return: 
            Tables -A list of tables residing in the database   
            
    '''
    conn = psycopg2.connect(
        database=database, 
        user='postgres', 
        password=Admin_Pass, 
        host='127.0.0.1', 
        port= '5434'
    )
    conn.autocommit = True
    # open connection
    
    cursor = conn.cursor()
    # Creating a cursor object
 
    cursor.execute("""SELECT table_name \
                           FROM information_schema.tables \
                           WHERE table_schema = 'public'""")
    Tables = [] 
    # empty list of tables to be filled
    for table in cursor.fetchall():
        Tables.append(table[0])
        # need the [0] to get the string from the tuple         
    cursor.close()
    # close the cursor
    conn.close()    
    # Closing the connection
    return Tables


def create_table(database):  
    
    '''    
    This function gets a list of tables 
    that are in the database 
    
        Args: 
            database - a string referencing the database containing the tables
        Return: 
            Tables -A list of tables residing in the database   
            
    '''
    conn = psycopg2.connect(
        database=database, 
        user='postgres', 
        password=Admin_Pass, 
        host='127.0.0.1', 
        port= '5434'
    )
    conn.autocommit = True
    # open connection
    
    cursor = conn.cursor()
    # Creating a cursor object
    
    print("Current list of tables in the database:")
    Table_List = print_table_list(database)
    
    table_name = input("What is the name of the new table? ")
    # Gets the name of the new table
    
    if table_name in Table_List:
        # check if the table is in the list already
        print("Table already in database ")
        
    else:        
        sql_command = sql.SQL("CREATE TABLE {table}()")\
                .format(table=sql.Identifier(table_name))
        # SQL command for creating new table
        cursor.execute(sql_command)
        print("Table \'%s\' created in database \'%s\'" %(table_name, database))
    
    choice = "yes"
    
    while choice.lower() in yes_choices:
        
        choice = input("\nAdd a column to table \'%s\'? " %table_name)
        if choice.lower() in yes_choices:
            
            add_column(table_name, database)
            
        choice = input("\nUpdate the primary key for table \'%s\'? " %(table_name ))
    
        if choice.lower() in yes_choices:
            
            set_primary_key(table_name, database)
            
        choice = input("\nAnything else for table \'%s\'? " %(table_name ))

    
    cursor.close()
    # close the cursor
    conn.close()    
    # Closing the connection
    
    print("Finished making table \'%s\'" %table_name) 
    print("Please use modify table function for future changes to this table" )
    return 


def add_column(table, database):
    
    '''
    This function adds a column to a specified table in a database, along with 
    its data type. Note: due to the limited need for datatypes in this assignment,
    only 3 datatypes are used here: varychar(n), numeric(p,s), and int. 
    
        Args: 
            database - a string referencing the database containing the tables
            table - a string referencing a table to add a column to 
        Return: 
           none           
    '''    
    conn = psycopg2.connect(
        database=database, 
        user='postgres', 
        password=Admin_Pass, 
        host='127.0.0.1', 
        port= '5434'
    )
    conn.autocommit = True
    # open connection
    cursor = conn.cursor()
    # Creating a cursor object
    
    SQL_Command = sql.SQL("SELECT * FROM {working_table}")\
            .format(working_table=sql.Identifier(table))
    # Establishing SQL Command    
    cursor.execute(SQL_Command, table)
    # Establishing cursor for the table picked by the user 
    column_names = [desc[0] for desc in cursor.description]
    # saving a list of the column names for the table picked by the user       
    go_back = True 
    # setting up for while loop below
    
    while go_back == True:
    
        go_back = False  
        # if go_back isn't changed to True, the loop should break
        # after this section is finished                
        print("What is the name of the new column? ")
        col_name = input("If not adding a column now, please type the word QUIT to end\n")
        
        if col_name.lower() == "quit":
            print("Quitting Now..")
            break

        # Gets the name of the new column
    
        if col_name in column_names:
            # check if the col_name already in the table
            print("\n\'%s\' already in table\n" %(col_name))
            print("Current column names for this table from left to right: \n")
            cnt = 1
            for col in column_names:
                print("%d) %s" %(cnt, col))
                cnt+=1    
        else:       
            print("\nChoose a data type for the column from these three datatypes")
        
            print("1) - varchar(n) ")
            print("2) - numeric(p,s)")
            print("3) - int\n")
               
            options = ["1","2","3", "back"] # setting up options list
            selection = ""  # setting blank string for selection
            
            while selection not in options:
                selection = input("Select 1, 2, or 3 and press enter, or type 'back' and press enter to go back.\n")
                
                if selection == 'back':  
                    print("going back..")
                    go_back = True                    
                elif selection == "1": 
                    print("Datatype \'varchar(n)\' selected ")
                    max = int(input("What are the maximum number of characters allowed? "))
                    sql_command = sql.SQL("ALTER TABLE {working_table} \
                                            ADD COLUMN {column} varchar({n})")\
                                            .format(working_table=sql.Identifier(table), \
                                                column = sql.Identifier(col_name),\
                                                n = sql.Literal(max)
                                            )                    
                    cursor.execute(sql_command)
                    print("\nColumn \'%s\' added to table \'%s\' with data type varchar(%s)\n" \
                          %(col_name,table,max))
                    column_names.append(col_name)
                    # adding new column to columns list
                    
                elif selection == "2": 
                    print("Datatype \'numeric(p,s)\' selected ")
                    p = input("How many digits are allowed in total?\
                    \nRemember to count those both left and right of the decimal point ")
                    s = input("How many digits to the right of the decimal are allowed? ")
                    sql_command = sql.SQL("ALTER TABLE {working_table} \
                                            ADD COLUMN {column} numeric({m},{n})")\
                                            .format(working_table=sql.Identifier(table), \
                                                column = sql.Identifier(col_name),\
                                                m = sql.Literal(p),
                                                n = sql.Literal(s)
                                            )                    
                    cursor.execute(sql_command)
                    print("\nColumn \'%s\' added to table \'%s\' with data type numeric(%s,%s)\n" \
                          %(col_name,table,p,s))
                    column_names.append(col_name)
                     # adding new column to columns list
                elif selection == "3":
                    print("Datatype \'int\' selected ")
                    sql_command = sql.SQL("ALTER TABLE {working_table} \
                                            ADD COLUMN {column} INT")\
                                            .format(working_table=sql.Identifier(table), \
                                                column = sql.Identifier(col_name)
                                            )                    
                    cursor.execute(sql_command)
                    print("\nColumn \'%s\' added to table \'%s\' with data type INT\n" \
                          %(col_name,table))
                    column_names.append(col_name)
                     # adding new column to columns list
                else:
                    print("Please select from the following options: ")
                             
            if go_back == True:
                # goes back to selecting a name for the column
                continue    
            print("Current column names for this table from left to right: \n")
            cnt = 1
            for col in column_names:
                print("%d) %s" %(cnt, col))
                cnt+=1             
    cursor.close()
    # close the cursor
    conn.close()    
    # Closing the connection
     
    return

For each table and for later steps, we will need to be able to select the columns we need from the tables, as well as set and update the primary key for each table.

In [17]:
def select_columnns(cols):
    
    '''
    This function takes in a list of columns and either returns a list of columns
    picked by the user, or returns the string *, which means the user wants to see all
    of the columns in the table
        
        Args: 
            cols - a list of columns in a table
        Return: 
            column_list - a list of strings referring to the columns in the table
            str(""*"") - if the user does not want to select only one column
            
    '''
    
    print("Would you like to pick any specific column(s) from the table to view? ")    
    choice = input("Type (y/n) and hit ENTER when finished ")
      
    if choice.lower() in yes_choices:
    
        column_list = []
        # initalizes empty string
        no_more = False   
        # sets variable to False to add more columns
        while len(column_list) == 0 or no_more == False :
            
            if len(column_list) != 0:
                print("\nColumn names already selected from this table: \n")
                cnt = 1
                for col in column_list:
                    print("%d) %s" %(cnt, col))
                    cnt+=1
            
            print("\nColumn names not selected yet for this table from left to right: \n")
            cnt = 1
            for col in cols:
                if col not in column_list:
                    print("%d) %s" %(cnt, col))
                cnt+=1
    
            print("\nPick a column from the list")
            print("\nYou may either the column's corresponding a number") 
            print("or type the column name.")
            choice = input()
            
            if choice.isnumeric() == True:
                # If the user picks the corresponding number for the column
                
                if int(choice) < 1 or int(choice) > len(cols):
                    
                    print("Invalid numeric choice\n")
                    
                else:
                        
                    cnt = 1
                    for col in cols:
                        if cnt == int(choice): 
                            if col not in column_list:
                                column_list.append(col)
                                print(col, "has been added to the list of columns to select")
                            else:
                                print("Column already previously selected")
                        cnt+=1
                
                    if len(column_list) == len(cols):
                    # In case the user selects all of the columns
                        print("\nList of columns is full - returning * instead of column list")
                        return "*" 
                                
                add_another = input("Add another column? ")
                if add_another != "yes" and add_another != "y" and add_another != "Y"\
                    and add_another != "YES" and add_another != "Yes":
                    print("\nFinal List of Columns")
                    print(column_list)
                    no_more = True
                    # leaving the function
                    
                
            elif choice in cols:
                # If the user picks the corresponding name for the column
                
                if choice not in column_list:
                    column_list.append(choice)
                    print(choice, "has been added to the list of columns to select")
                else:
                    print("Column already previously selected")
                    
                if len(column_list) == len(cols):
                # In case the user selects all of the columns
                    print("\nList of columns is full - returning * instead of column list")
                    return "*" 
                    
                add_another = input("Add another column? ")
                if add_another != "yes" and add_another != "y" and add_another != "Y"\
                    and add_another != "YES" and add_another != "Yes":
                    print("\nFinal List of Columns")
                    print(column_list)
                    no_more = True
                    # leaving the function
                                       
            else:
                # If the user picks something that's not a 
                # column name or its matching number
                
                print("Column not found by name or number")
                print("Would you like to add another column to the list? ")
    
                choice = input("Type (y/n) and hit ENTER when finished ")
                if choice != "yes" and choice != "y" and choice != "Y"\
                    and choice != "YES" and choice != "Yes":              
                    if len(column_list) == 0:                
                        return "*"                          
                    else:   
                        print("\nFinal List of Columns")
                        print(column_list)
                        return column_list                                
        return column_list
    
    else:
        return "*"

def set_primary_key(table, database):
    
    '''
    This function sets one or more columns as the primary key for a table. 
    
        Args: 
            database - a string referencing the database containing the tables
            table - a string referencing a table to add a column to 
        Return: 
           none           
    '''  
    conn = psycopg2.connect(
        database=database, 
        user='postgres', 
        password=Admin_Pass, 
        host='127.0.0.1', 
        port= '5434'
    )
    conn.autocommit = True
    # open connection
    cursor = conn.cursor()
    # Creating a cursor object
    SQL_Command = sql.SQL("SELECT * FROM {working_table}")\
            .format(working_table=sql.Identifier(table))
    # Establishing SQL Command    
    cursor.execute(SQL_Command, table)
    # Establishing cursor for the table picked by the user 
    column_names = [desc[0] for desc in cursor.description]
    # saving a list of the column names for the table picked by the user       
    current_primary_keys = get_primary_keys(table, database)
    # getting current list of primary keys for the table in the database
    
    if len(current_primary_keys) > 0:
        # lets user know that there are already primary keys and gives option to
        # not change them
        
        print("Primary keys already set for this table.")
        print("Curren column(s) set as the primary key")
        print(current_primary_keys)
        choice = input("Do you still want to change the primary keys for this table ? ")
        if choice not in yes_choices:
            return
    
    print("Selecting list of columns to use for primary key: \n")
    primary_key_cols = select_columnns(column_names)
    # getting selection of columns to use as the primary key
    
    if primary_key_cols == "*":
        # to ensure that the primary key is listed as all columns, not *
        primary_key_cols = column_names
            
    if len(current_primary_keys) > 0: 
        # different command needed to replace primary key 
        
        pk_string = table + "_pkey" 
        # auto-generated constraint name for table primary key - needed for statement below
        
        SQL_Command = sql.SQL("ALTER TABLE {working_table} \
                               DROP CONSTRAINT {constraint},\
                               ADD PRIMARY KEY ({columns})").\
                      format( 
                            columns = sql.SQL(', ')\
                             .join(sql.Identifier(col) \
                             for col in primary_key_cols), \
                             # these join statements are needed
                             # in order to use the columns selected 
                             # im the SQL statement
                        working_table=sql.Identifier(table),
                        constraint=sql.Identifier(pk_string)
                         # puts the name of the table and constraint into command
                    )        
    else:     
        # if primary key hasn't already been set
        SQL_Command = sql.SQL("ALTER TABLE {working_table} \
                               ADD PRIMARY KEY ({columns})").\
                      format( 
                            columns = sql.SQL(', ')\
                             .join(sql.Identifier(col) \
                             for col in primary_key_cols), \
                             # these join statements are needed
                             # in order to use the columns selected 
                             # im the SQL statement
                        working_table=sql.Identifier(table)
                         # puts the name of the table into command
                    )
    
    # Using format that cursor.execute can read using sql package
    cursor.execute(SQL_Command, table )
    print("\nNew primary key for table \'%s\':" %(table))
    print(primary_key_cols)
    
    cursor.close()
    # close the cursor
    conn.close()    
    # Closing the connection
    
    return

def get_primary_keys(table, database):
    '''
    This function takes in a table and a database and returns a list of columns that are listed as
    as the primary key for the table
        Args: 
            database - a string referencing the database containing the tables
            table - a string referencing a table to add a column to 
        Return: 
            primary_keys - a list of strings referring to the columns in the table
            that are the current primary keys in the table
            
    '''
    conn = psycopg2.connect(
        database=database, 
        user='postgres', 
        password=Admin_Pass, 
        host='127.0.0.1', 
        port= '5434'
    )
    conn.autocommit = True
    # open connection
    cursor = conn.cursor()
    # Creating a cursor object
  
    SQL_Command = sql.SQL("    SELECT c.column_name \
                               FROM information_schema.key_column_usage AS c \
                               LEFT JOIN information_schema.table_constraints AS t \
                               ON t.constraint_name = c.constraint_name \
                               WHERE t.table_name = {working_table} \
                               AND t.constraint_type = 'PRIMARY KEY'").\
                               format( working_table=sql.Literal(table)
                                 # puts the name of the table into command
                                 # also needed sql.Literal here instead of sql.Identifier
                    )
    cursor.execute(SQL_Command, table )
    primary_keys_raw = cursor.fetchall()  
    # this gets a list, but they are all in parentheses, so next step needed
    
    primary_keys = [] # initialize empty list
    
    for i in range(len(primary_keys_raw)):
        primary_keys.append(primary_keys_raw[i][0])
        # gets column names out of parenetheses
   
    cursor.close()
    # close the cursor
    conn.close()    
    # Closing the connection
    
    return primary_keys

def view_table(table,database):
    
    '''
        This function takes in a table and a database and displays the data in a table
        format along with the row numbers
    
        Args: 
            database - a string referencing the database containing the tables
            table - a string referencing a table to add a column to 
            
        Return: 
            none
    
    '''    
    conn = psycopg2.connect(
        database=database, 
        user='postgres', 
        password=Admin_Pas, 
        host='127.0.0.1', 
        port= '5434'
    )
    conn.autocommit = True
    # open connection
    cursor = conn.cursor()
    # Creating a cursor object
    
    primary_keys = get_primary_keys(table, database)
    
    SQL_Command = sql.SQL("SELECT * FROM {working_table} ORDER BY {keys}")\
                            .format(working_table=sql.Identifier(table),
                               keys = sql.SQL(', ').join(sql.Identifier(key)\
                                 for key in primary_keys))
                             # these join statements are needed
                             # in order to use the columns selected 
                             # im the SQL statement))
    # Establishing SQL Command    
    cursor.execute(SQL_Command, table)
    # Establishing cursor for the table picked by the user 
    column_names = [desc[0] for desc in cursor.description]
    # saving a list of the column names for the table picked by the user 
    results = cursor.fetchall()
    # gets all of the data from the table
    rowIDs = []     
    # initialize a list of the row ids
    row = 1  
    for result in results:       
        rowIDs.append(row)
        row = row+1
        # starts from 1 instead of zero for rows 
    print(tabulate(results, headers= column_names, tablefmt='psql', showindex=rowIDs))
    # displays the table
    cursor.close()
    # close the cursor
    conn.close()    
    # Closing the connection     
    return

Now to create any tables that have not been created already. For the purposes of this assignment, the data types for the columns will be restricted to the three types needed for our tables, which are **varchar(n)**, **numeric(p,s)** and **int**.

In [ ]:
create_table(database_name)
# Creating 'Product' table with columns (maker, model, type) w/ model as primary key

In [ ]:
create_table(database_name) 
# Creating 'PC' table with columns (model, speed, ram, hd, price) w/ model as primary key

In [ ]:
create_table(database_name)
# Creating 'Laptop' table with columns (model, speed, ram, hd, screen, price) w/ model as primary key

In [ ]:
create_table(database_name)
# Creating 'Printer' table with columns (model, color, type, price) w/ model as primary key

In [ ]:
def modify_table(table,database,data=[]):
    
    '''
    This function takes in a table and a database and modifies the table according on the user selection
    
        Args: 
            database - a string referencing the database containing the tables
            table - a string referencing a table to add a column to 
        Return: 
            none
            
    '''  
    if data == []:
        manually_add = True
    else:
        manually_add = False
    # Determines whether the user plans to manually input the data or not
    
    print("\nChoose an action to modify the table")
        
    print("1) - Insert Data ")
    print("2) - Remove Rows")
    print("3) - Add a Column\n")
    print("4) - Quit\n")
               
    options = ["1","2","3","4", "back"] # setting up options list
    selection = ""  # setting blank string for selection
            
    while selection not in options:
        selection = input("Select 1, 2, 3, or 4 and press enter. ")
        
        if selection == "1" and manually_add == True:
            add_data_into_table(table,database,True)
        elif selection == "1" and manually_add == False:
            add_data_into_table(table,database, False, data)
        elif selection == "2":
            remove_rows_from_table()
        elif selection == "3":
            add_column(table, database)
        elif selection == "4":
            print("Quitting now..")
            break
        else:
            print("Please select one of the options listed and press enter.")   
    return


def add_data_into_table(table,database, manually_add = True, data=[]):
    
    '''
    This function takes in a table and a database and adds data into the table
    either manually, or using a list provided
    
        Args: 
            database - a string referencing the database containing the tables
            table - a string referencing a table to add a column to 
            
        Defaults:
            manually_add - A boolean value specifying whether to add the data manually or not
            data - a list of values to add to the table. The data must match the datatype
                    and the columns exactly. The default is empty
        Return: 
            none
            
    '''
    
    conn = psycopg2.connect(
        database=database, 
        user='postgres', 
        password=Admin_Pass, 
        host='127.0.0.1', 
        port= '5434'
    )
    conn.autocommit = True
    # open connection
    cursor = conn.cursor()
    # Creating a cursor object
    
    SQL_Command = sql.SQL("SELECT * FROM {working_table}")\
            .format(working_table=sql.Identifier(table))
    # Establishing SQL Command    
    cursor.execute(SQL_Command, table)
    # Establishing cursor for the table picked by the user 
    column_names = [desc[0] for desc in cursor.description]
    # saving a list of the column names for the table picked by the user 
    
    data_num_rows = len(data)
    data_num_cols = len(data[0])
    
    for i in range(data_num_rows):
            
        sql_command = sql.SQL("INSERT INTO{working_table} ({columns}) VALUES ({values})")\
                        .format( 
                            columns = sql.SQL(', ')\
                             .join(sql.Identifier(col) \
                             for col in column_names), \
                             # these join statements are needed
                             # in order to use the columns selected 
                             # im the SQL statement
                            values = sql.SQL(', ')\
                             .join(sql.Literal(value) \
                             for value in data[i]), \
                             # these join statements are needed
                             # in order to use the values selected 
                             # im the SQL statement
                            working_table=sql.Identifier(table),
                             # puts the name of the table into command
                    )             
        cursor.execute(sql_command, data[i])
        print(data[i], "added to table \'%s\'" %table)
        
    cursor.close()
    # close the cursor
    conn.close()    
    # Closing the connection   
    return

def remove_rows_from_table(table,database):
    '''
        This function takes in a table and a database and removes rows into the table
    specified by the user
    
        Args: 
            database - a string referencing the database containing the tables
            table - a string referencing a table to add a column to 
            
        Return: 
            none   
    '''
    view_table(table,database) 
    # displays the table first in order for the user to know which rows to remove
    primary_keys = get_primary_keys(table, database)
    # gets the primary keys for use in the function below
    print("PRIMARY KEYS ARE -", primary_keys)
    
    conn = psycopg2.connect(
        database=database, 
        user='postgres', 
        password=Admin_Pass, 
        host='127.0.0.1', 
        port= '5434'
    )
    conn.autocommit = True
    # open connection
    cursor = conn.cursor()
    # Creating a cursor object
    
    SQL_Command = sql.SQL("SELECT * FROM {working_table}")\
            .format(working_table=sql.Identifier(table))
    # Establishing SQL Command    
    cursor.execute(SQL_Command, table)
    # Establishing cursor for the table picked by the user 
    column_names = [desc[0] for desc in cursor.description]
    # saving a list of the column names for the table picked by the user 
    
    rows_to_remove = [] # initialize list of rows to remove
    
    print("Input a list of row numbers to remove from the table.")
    remove_list = input("Separate each row number with a comma. To go back, type QUIT \n")
    if remove_list == '' or remove_list.lower() == "quit":
        print("Quitting now..")
        return
    else:
        num = str()
        # initialize an empty string to store the numbers
        cnt = 1
        # assume the string has length of 1
        for char in remove_list:
            # loops through each character in the string
            if cnt == len(remove_list) and remove_list[-1].isnumeric() == True:
                # to deal with last character in the string
                # and ignoring it if it's not numeric
                rows_to_remove.append(int(num+remove_list[-1]))
                # adds number plus the last char in string 
            elif char.isnumeric() == False and len(num) != 0:
                # deals with next break after a new integer has been started
                # so the length of num is not 0 here
                rows_to_remove.append(int(num))
                # add integer to list
                num = str()
                # reset num list for next number in the list
            elif char.isnumeric() == False:
                # deals with empty spaces and other characters
                # that aren't numbers, keeps num from adding chars
                # until an actual number is reached
                num = num
            else:
                # adds numeric characters until
                # something that is not a numeric char
                # is reached
                num = num+char
            cnt = cnt +1
    
    rows_to_remove.sort() # sorts the list in ascending order
    
    print("Rows to be deleted", rows_to_remove)
    print("Data in all these rows will be deleted, are you sure you want to delete?")
    choice = input("Type yes and hit enter to confirm deletion")
    if choice not in yes_choices:
          print("Rows were not deleted")
          return
    
    cnt = 0 # initializing count to deal with changing index below
                         
    for row in rows_to_remove:
    # goes through each row that the user wants to delete                          
           
            print("Removing ROW %s " %row)           
            sql_command = sql.SQL("SELECT * \
                                   FROM {working_table} \
                                   ORDER BY {keys}\
                                   LIMIT 1 \
                                   OFFSET {index}")\
                                    .format(working_table=sql.Identifier(table),\
                                        keys = sql.SQL(', ').\
                                     join(sql.Identifier(key) \
                                          for key in primary_keys),\
                                       index = sql.Literal(row-1-cnt)  ) 
                                        # every time a deletion happens
                                        # there are 1 less rows in the table
                                        # so cnt needs to be subtracted too
            
            cursor.execute(sql_command)
            deleted_row = cursor.fetchall()
            print(deleted_row, "removed from table \'%s\'" %table)

            ### DOES THIS COMMAND ABOVE FIRST
            ### SO THAT THE USER CAN SEE WHAT DATA IS BEING
            ## REMOVED FROM THE TABLE

            sql_command = sql.SQL("DELETE FROM {working_table}\
                                  WHERE {keys} = (SELECT {keys} \
                                                   FROM {working_table} \
                                                   ORDER BY {keys}\
                                                   LIMIT 1 \
                                                   OFFSET {index})\
                                  RETURNING *")\
                            .format(working_table=sql.Identifier(table),\
                                        keys = sql.SQL(', ').\
                                     join(sql.Identifier(key) \
                                          for key in primary_keys),\
                                       index = sql.Literal(row-1-cnt)  ) 
                                        # IMPORTANT !!!
                                        # every time a deletion happens
                                        # there are 1 less rows in the table
                                        # so cnt needs to be subtracted too
                            
            # NOTE -  The SQL Command here deletes the rows 
            # that contain the primary key value contained in the row specified
            # Since the primary key is unique
            # only the row we want to remove is deleted
            cursor.execute(sql_command)
            cnt+=1
            
    cursor.close()
    # close the cursor
    conn.close()    
    # Closing the connection   
    return

### Here is the data for the tables provided in pages 53 and 54 of the book "Database Systems: The Complete Book, 2nd Edition"

In [ ]:
Product_Table_Data = [('A',1001,'pc'),
                      ('A',1002,'pc'),
                      ('A',1003,'pc'),
                      ('A',2004,'laptop'),
                      ('A',2005,'laptop'),
                      ('A',2006,'laptop'),
                      ('B',1004, 'pc'),
                      ('B',1005, 'pc'),
                      ('B',1006, 'pc'),
                      ('B',2007, 'laptop'),
                      ('C',1007, 'pc'),
                      ('D',1008, 'pc'),
                      ('D',1009, 'pc'),
                      ('D',1010, 'pc'),
                      ('D',3004, 'printer'),
                      ('D',3005, 'printer'),
                      ('E',1011, 'pc'),
                      ('E',1012, 'pc'),
                      ('E',1013, 'pc'),
                      ('E',2001, 'laptop'),
                      ('E',2002, 'laptop'),
                      ('E',2003, 'laptop'),
                      ('E',3001, 'printer'),
                      ('E',3002, 'printer'),
                      ('E',3003, 'printer'),
                      ('F',2008,'laptop'),
                      ('F',2009,'laptop'),
                      ('G',2010,'laptop'),
                      ('H', 3006, 'printer'),
                      ('H', 3007, 'printer'),           
                     ]

PC_Table_Data = [(1001,2.66,1024,250,2114),
                 (1002,2.10,512,250,995),
                 (1003,1.42,512,80,478),
                 (1004,2.80,1024,250,649),
                 (1005,3.20,512,250,630),
                 (1006,3.20,1024,320,1049),
                 (1007,2.20,1024,200,510),
                 (1008,2.20,2048,250,770),
                 (1009,2.00,1024,250,650),
                 (1010,2.80,2048,300,770),
                 (1011,1.86,2048,160,959),
                 (1012,2.80,1024,160,649),
                 (1013,3.06,512,80,529),
                     ]

Laptop_Table_Data = [(2001,2.00,2048,240,20.1,3673),
                     (2002,1.73,1024,80,17.0,949),
                     (2003,1.80,512,60,15.4,549),
                     (2004,2.00, 512, 60, 13.3, 1150),
                     (2005,2.16,1024,120,17.0,2500),
                     (2006,2.00,2048,80,15.4,1700),
                     (2007,1.83,1024,120,13.3,1429),
                     (2008,1.60,1024,100,15.4,900),
                     (2009,1.60,512,80,14.1,680),
                     (2010,2.00,2048,160,15.4,2300),    
                    ]

Printer_Table_Data = [(3001,'true','ink-jet',99),
                      (3002,'false','laser',239),
                      (3003,'true','laser',899),
                      (3004,'true','ink-jet',120),
                      (3005,'false','laser',120),
                      (3006,'true','ink-jet',100),
                      (3007,'true','laser',200),   
                      ]


Now to fill in the tables with the data above:

**NOTE** Running this multiple times will cause errors, due to the data already being in the table and violating the primary key constraints

In [ ]:
add_data_into_table('Product',"Assignment 4",data=Product_Table_Data)
add_data_into_table('PC',"Assignment 4",data=PC_Table_Data)
add_data_into_table('Laptop',"Assignment 4",data=Laptop_Table_Data)
add_data_into_table('Printer',"Assignment 4",data=Printer_Table_Data)

Now to ensure that the functions created above are working. Let's test the **view_table** function and **remove_rows_from_table** functions first

In [ ]:
view_table('Printer',"Assignment 4")
# Looking at the table currently

remove_rows_from_table('Printer',"Assignment 4")
# testing remove rows function using rows 1,3,and 4

print("Here is the table after removing rows 1, 3 and 4")
view_table('Printer',"Assignment 4")
# viewing table after removing rows


Now to test addding data back into the table using the **add_data_into_table()** function again



In [ ]:
Removed_Printer_Rows = [(3001, 'true', 'ink-jet', 99),
                        (3003, 'true', 'laser', 899),
                        (3004, 'true', 'ink-jet', 120)]
# The three rows that were removed above
add_data_into_table('Printer',"Assignment 4",data=Removed_Printer_Rows)
# addding the data back
print("Here is the restored table sorted by primary key")
view_table('Printer',"Assignment 4")
# viewing table after adding back rows


### 1. Add an assertion that the average speed of PCs may not be less than a certain value, i.e., 1Mhz.

Per this documentation, 

https://www.postgresql.org/docs/7.2/sql-createtable.html

PostgreSQL does not allow assertions to be added, so the questions on assertions will be written out but not implemented.

Using this format for answer -  **CREATE ASSERTION** name **CHECK** ( condition )

Setting the speed to an arbitray minimum speed of **1.5**  
The conmnand to add an assertion would be - 

**DROP ASSERTION IF EXISTS** Avg_PC_Speed_Not_Less_than_1.5   
**CREATE ASSERTION** Avg_PC_Speed_Not_Less_than_1.5  
**CHECK**( NOT EXISTS  

            (SELECT * FROM "PC"
            WHERE (SELECT AVG(speed) FROM "PC") < 1.5 )
        )


### 2. Add an assertion that the average speed of PCs may not be than average speed of Laptops.

This question doesn't specifiy greater or less than, so we will go with an assertion that the average speed of PCs may not be less than the average speed of Laptops

**DROP ASSERTION IF EXISTS** Avg_PC_Speed_Must_Be_Greater_Than_Avg_Laptop_Speed   
**CREATE ASSERTION** Avg_PC_Speed_Must_Be_Greater_Than_Avg_Laptop_Speed  
**CHECK**( NOT EXISTS  

            (SELECT * FROM "Laptop"
            WHERE (SELECT AVG(speed) FROM "PC") < 
            (SELECT AVG(speed) FROM "Laptop")
        )


### 3. Generalize assertion 1 so the DBA can assert it for any speed whenever it is needed.

Here we drop the previous assertion for the minimum speed after getting the new one from the DBA, and add the new assertion with the updated speed

In [ ]:
Min_Speed = input("Input the minimum speed. The Average PC speed must not be less than this\n")

**DROP ASSERTION IF EXISTS** Min_PC_Speed  
**CREATE ASSERTION** Min_PC_Speed  
**CHECK**( NOT EXISTS

          (SELECT * FROM "PC"
          WHERE (SELECT AVG(speed) FROM "PC") < Min_Speed )
    )

### 4. Add a trigger to prevent that a maker can not produce both PCs and Laptops, only can produce one.

Before creating a trigger, a trigger function needs to be created. Here is the basic format below

In [ ]:
# Basic Format for creating a trigger function

'''

CREATE FUNCTION trigger_function() 
   RETURNS TRIGGER 
   LANGUAGE PLPGSQL
AS $$
BEGIN
   -- trigger logic
END;
$$

'''

Using that format, we can make a trigger function that removes the row if the row being added causes the maker to make both pcs and laptops, and not one or the other. The code is listed below.


In [ ]:
'''
CREATE OR REPLACE FUNCTION  PCs_Or_Laptops()
  RETURNS TRIGGER 
  LANGUAGE PLPGSQL
  AS
$$
BEGIN
	IF NEW.type = 'pc' AND NEW.maker IN (SELECT maker FROM "Product" where "type" = 'laptop') 
	OR NEW.type = 'laptop' AND NEW.maker IN (SELECT maker FROM "Product" where "type" = 'pc') THEN
	
		DELETE FROM "Product"
		WHERE maker = NEW.maker AND "type" = NEW.type;
		
	END IF;
	RETURN NULL;
END
$$

'''

And now that the trigger function is created, we can create the trigger to take place after the update or insert of new data

In [ ]:
''''
-- Trigger: Cannot make both PCs and Laptops

DROP TRIGGER IF EXISTS "Cannot make both PCs and Laptops" ON public."Product";

CREATE CONSTRAINT TRIGGER "Cannot make both PCs and Laptops"
    AFTER INSERT OR UPDATE OF maker, type
    ON public."Product"
    FOR EACH ROW
    EXECUTE FUNCTION public.pcs_or_laptops();
    
'''

Lastly, we will attempt to add this trigger via EmbeddedSQL

In [ ]:
def add_trigger_to_database(table,database):
    
    '''
    This function adds a trigger to the 'Product' table that prevents
    it from allowing a maker that makes both PCs and Laptops. 
    NOTE - This function is very specific to the tables listed in the assignment
    and not very good for general use outside of this assignment. However
    it does successfully implement a moderately complex trigger function
    and trigger
    
    Args: 
        database - a string referencing the database containing the tables
        table - a string referencing a table to add a column to            
    Return: 
        none 
   
    '''
    
    column1 = 'type'
    column2 = 'maker'
    
    conn = psycopg2.connect(
        database=database, 
        user='postgres', 
        password=Admin_Pass, 
        host='127.0.0.1', 
        port= '5434'
    )
    conn.autocommit = True
    # open connection
    cursor = conn.cursor()
    # Creating a cursor object
    
    SQL_Command = sql.SQL("CREATE OR REPLACE FUNCTION PCs_Or_Laptops()\n"\
                          "RETURNS TRIGGER\n" \
                          "LANGUAGE PLPGSQL \
                          AS $$"\
                              "BEGIN \n"\
                                 "\tIF NEW.type = 'pc' AND NEW.maker IN \n"\
                                 "\t(SELECT {col2} FROM {working_table} where {col1} = 'laptop')\n "\
                                 "\tOR NEW.type = 'laptop' AND NEW.maker IN \n"\
                                 "\t(SELECT {col2} FROM {working_table} where {col1} = 'pc') \n"\
                                 "\tTHEN \n"\
                                     "\t\tDELETE FROM {working_table}\n"\
                                     "\t\tWHERE {col2} = NEW.{col2} AND {col1} = NEW.{col1};\n"\
                                 "\tEND IF; \n"\
                                "\tRETURN NULL; \n"\
                              "END\n"\
                             "$$")\
                                 .format(working_table=sql.Identifier(table),\
                                         col1=sql.Identifier(column1),\
                                         col2=sql.Identifier(column2)\
                                        )
                         
    # Establishing SQL Command for the creation of the function 
    cursor.execute(SQL_Command, table)
    # creating trigger function
    print("Trigger function \'PCs_Or_Laptops(\'') added ")
                                                   
    SQL_Command = sql.SQL("DROP TRIGGER IF EXISTS {title2} ON {working_table};\
                           CREATE CONSTRAINT TRIGGER {title2}\
                           AFTER INSERT OR UPDATE OF {col2}, {col1} ON {working_table}\
                           FOR EACH ROW\
                           EXECUTE FUNCTION PCs_Or_Laptops();")\
                            .format(working_table=sql.Identifier(table),\
                                         col1=sql.Identifier(column1),\
                                         col2=sql.Identifier(column2),\
                                         title2=sql.Identifier("Cannot make both PCs and Laptops"),\
                                        )
                         
    # Establishing SQL Command for the creation of the trigger 
    cursor.execute(SQL_Command, table)
    # creating trigger          
    print("Trigger \'Cannot make both PCs and Laptops\'' added ")
    cursor.close()
    # close the cursor
    conn.close()    
    # Closing the connection  
    return

In [ ]:
add_trigger_to_database("Product","Assignment 4")

It appears that the trigger was succesfully added. Below shows the code that was added for the trigger in PostgreSQL via the embedded SQL command above. Now to remove all of the rows, then add back the data, and seee how the trigger performs

In [ ]:
'''
BEGIN 
	IF NEW.type = 'pc' AND NEW.maker IN 
	(SELECT "maker" FROM "Product" where "type" = 'laptop')
 	OR NEW.type = 'laptop' AND NEW.maker IN 
	(SELECT "maker" FROM "Product" where "type" = 'pc') 
	THEN 
		DELETE FROM "Product"
		WHERE "maker" = NEW."maker" AND "type" = NEW."type";
	END IF; 
	RETURN NULL; 
END

'''

In [ ]:
remove_rows_from_table('Product',"Assignment 4")
# removing all rows this time

In [ ]:
add_data_into_table('Product',"Assignment 4",data=Product_Table_Data)
view_table('Product',"Assignment 4")

So the trigger function was successfully implemented, as we can see that when we filled up the table, the trigger function eliminated the rows with the different types of computer (ie; if a row with a maker making a laptop was added and the maker already made PCS, the row was deleted and vice versa for the maker making a PC and trying to add a row with the maker making a laptop.

### 5. Implement Assertion 2 as a trigger.

Again, this question doesn't specifiy greater or less than, so we will go with a trigger that prevents the the average speed of PCs from being less than the average speed of Laptops

The assertion above was this

**DROP ASSERTION IF EXISTS** Avg_PC_Speed_Must_Be_Greater_Than_Avg_Laptop_Speed   
**CREATE ASSERTION** Avg_PC_Speed_Must_Be_Greater_Than_Avg_Laptop_Speed  
**CHECK**( NOT EXISTS  

            (SELECT * FROM "Laptop"
            WHERE (SELECT AVG(speed) FROM "PC") < 
            (SELECT AVG(speed) FROM "Laptop")
        )

So now we need to make this into a trigger somehow. Below shows the code for the trigger function

In [ ]:
'''
############ PC Trigger ############

BEGIN 
	IF EXISTS  (SELECT * FROM "Laptop" 
	WHERE (SELECT AVG(speed) FROM "PC") < 
	(SELECT AVG(speed) FROM "Laptop")) 
	THEN
		DELETE FROM "PC" 
		WHERE model = NEW.model;
	END IF;
	RETURN NULL;
END

############ Laptop Trigger ##########

BEGIN 
	IF EXISTS  (SELECT * FROM "Laptop" 
	WHERE (SELECT AVG(speed) FROM "PC") < 
	(SELECT AVG(speed) FROM "Laptop")) 
	THEN
		DELETE FROM "Laptop" 
		WHERE model = NEW.model;
	END IF;
	RETURN NULL;
END

'''

We will try to use the same function for triggers for both tables PC and Laptop

In [ ]:
def add_two_triggers_to_database(database):
    
    '''
    This function allows for a trigger function to work for either our Laptop
    Table or the PC table
    
    Args: 
        database - a string referencing the database containing the tables
        table - a string referencing a table to add a column to            
    Return: 
        none 
   
    '''
    column1 = "model"
    column2 = "speed"
    
    conn = psycopg2.connect(
        database=database,    
        user='postgres', 
        password=Admin_Pass, 
        host='127.0.0.1', 
        port= '5434'
    )
    conn.autocommit = True
    # open connection
    cursor = conn.cursor()
    # Creating a cursor object
    
    ########################################################################
    # ADDING TRIGGER CONDITIONS TO PC, WILL ADD LAPTOP SEPARATELY BELOW
    ########################################################################
    
    SQL_Command = sql.SQL("CREATE OR REPLACE FUNCTION PC_TRIGGER_FUNCTION()\n"\
                          "RETURNS TRIGGER\n" \
                          "LANGUAGE PLPGSQL \
                          AS $$"\
                              "BEGIN \n"\
                                 "\tIF EXISTS  (SELECT * FROM {Laptop} \n"\
                                 "\tWHERE (SELECT AVG(speed) FROM {PC}) < \n"\
                                 "\t(SELECT AVG(speed) FROM {Laptop})) \n"\
                                 "\tTHEN\n"\
                                        "\t\tDELETE FROM {PC} \n"\
                                        "\t\tWHERE model = NEW.model;\n"\
                                 "\tEND IF;\n"\
                                 "\tRETURN NULL;\n"\
                              "END"\
                             "$$")\
                                 .format(working_table=sql.Identifier("PC"),\
                                         Laptop = sql.Identifier('Laptop'),\
                                         PC = sql.Identifier('PC')
                                        )
                         
    # Establishing SQL Command for the creation of the function 
    cursor.execute(SQL_Command)
    # creating trigger function

    print("Trigger function \'PC_TRIGGER_FUNCTION()(\') added ")
                                                   
    SQL_Command = sql.SQL("DROP TRIGGER IF EXISTS {title2} ON {working_table};\
                           CREATE CONSTRAINT TRIGGER {title2}\
                           AFTER INSERT OR UPDATE OF {col2}, {col1} ON {working_table}\
                           FOR EACH ROW\
                           EXECUTE FUNCTION PC_TRIGGER_FUNCTION();")\
                            .format(working_table=sql.Identifier("PC"),\
                                         col1=sql.Identifier(column1),\
                                         col2=sql.Identifier(column2),\
                                         title2=sql.Identifier\
                                    ("Avg_PC_Speed_Must_Be_Greater_Than_Avg_Laptop_Speed_PC"),\
                                        )
                         
    # Establishing SQL Command for the creation of the trigger 
    cursor.execute(SQL_Command)
    # creating trigger          
    print("Trigger \'Avg_PC_Speed_Must_Be_Greater_Than_Avg_Laptop_Speed_P\'' added to PC table ")
    
    ########################################################################
    # ADDING SAME TRIGGER CONDITIONS TO LAPTOP, ONLY DIFFERENCE IS THE TABLE
    ########################################################################
    
    
    SQL_Command = sql.SQL("CREATE OR REPLACE FUNCTION LAPTOP_TRIGGER_FUNCTION()\n"\
                          "RETURNS TRIGGER\n" \
                          "LANGUAGE PLPGSQL \
                          AS $$"\
                              "BEGIN \n"\
                                 "\tIF EXISTS  (SELECT * FROM {Laptop} \n"\
                                 "\tWHERE (SELECT AVG(speed) FROM {PC}) < \n"\
                                 "\t(SELECT AVG(speed) FROM {Laptop})) \n"\
                                 "\tTHEN\n"\
                                        "\t\tDELETE FROM {Laptop} \n"\
                                        "\t\tWHERE model = NEW.model;\n"\
                                 "\tEND IF;\n"\
                                 "\tRETURN NULL;\n"\
                              "END"\
                             "$$")\
                                 .format(working_table=sql.Identifier("Laptop"),\
                                         Laptop = sql.Identifier('Laptop'),\
                                         PC = sql.Identifier('PC')
                                        )
    
    # Establishing SQL Command for the creation of the function 
    cursor.execute(SQL_Command)
    # creating trigger function

    print("Trigger function \'Laptop_TRIGGER_FUNCTION()(\') added ")
                                                   
    SQL_Command = sql.SQL("DROP TRIGGER IF EXISTS {title2} ON {working_table};\
                           CREATE CONSTRAINT TRIGGER {title2}\
                           AFTER INSERT OR UPDATE OF {col2}, {col1} ON {working_table}\
                           FOR EACH ROW\
                           EXECUTE FUNCTION Laptop_TRIGGER_FUNCTION();")\
                            .format(working_table=sql.Identifier("Laptop"),\
                                         col1=sql.Identifier(column1),\
                                         col2=sql.Identifier(column2),\
                                         title2=sql.Identifier\
                                    ("Avg_PC_Speed_Must_Be_Greater_Than_Avg_Laptop_Speed_L"),\
                                        )
                         
    # Establishing SQL Command for the creation of the trigger 
    cursor.execute(SQL_Command)
    # creating trigger          
    print("Trigger \'Avg_PC_Speed_Must_Be_Greater_Than_Avg_Laptop_Speed_L\'' added to Laptop table ")
    cursor.close()
    # close the cursor
    conn.close()    
    # Closing the connection  
    return

In [ ]:
add_two_triggers_to_database("Assignment 4")

The trigger function appears to have worked but it should be tested to be safe. Let's remove some rows from the tables to be sure. We'll remove all of the rows that don't have a speed of 2 in the laptop table first.

In [ ]:
remove_rows_from_table('Laptop',"Assignment 4")
# removing rows 2, 3, 5, 7, 8, 9 when the table is full

In [ ]:
view_table('Laptop',"Assignment 4")

We know just from looking that that the average speed is 2 for Laptop. Now to remove some PC rows and then add some data that should cause the trigger to act.

In [ ]:
remove_rows_from_table('PC',"Assignment 4")
# removing rows from PC table
# When table is full removing rows 1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13

In [ ]:
view_table('PC',"Assignment 4")

The average speed for the table now is also 2. Now let's add a row with a low speed to the PC table. IF the trigger is working, it should delete that row

In [ ]:
Low_PC_Speeds = [(1003, 1.42, 512, 80, 478),
                 (1011, 1.86, 2048, 160, 959) ] 

add_data_into_table('PC',"Assignment 4",data=Low_PC_Speeds)

In [ ]:
view_table('PC',"Assignment 4")

The trigger worked as expected. The add_data_into_table() function would have normally added the rows in without any problem, and there were no error messages. However, the trigger prevented the addition of the row due it causing the average speed of PCs to go below **2**, which is the average speed of Laptops currently. Now to try the opposite and add higher speeds to the Laptop table.

In [ ]:
view_table('Laptop',"Assignment 4")

In [ ]:
High_Laptop_Speeds = [(2005, 2.16, 1024, 120, 17.0, 2500)]
add_data_into_table("Laptop","Assignment 4",data=High_Laptop_Speeds)

If the trigger worked for Laptop as well, we will have the same table

In [ ]:
view_table('Laptop',"Assignment 4")

The trigger also prevented the higher amounts from being added as well, so the triggers are working as expected

### 6. Write an embedded SQL program in a language of your choice - list the PCs that are faster than the average speed of laptops

We can use this SQL query inside of Python to get our answer

**SELECT** * **FROM** "PC"  
**WHERE** speed > (**SELECT AVG**(speed) **FROM** "Laptop")  


and use it in the code above from the **view_table()** function slighltly modified
to fit the query

Let's make two new tables with no constraints PC2 and Laptop2, in order to finsih this last query

In [ ]:
create_table("Assignment 4")

In [ ]:
create_table("Assignment 4")

In [ ]:
add_data_into_table('PC2',"Assignment 4",data=PC_Table_Data)
add_data_into_table('Laptop2',"Assignment 4",data=Laptop_Table_Data)

In [ ]:
view_table('PC2',"Assignment 4")
view_table('Laptop2',"Assignment 4")

We now have the same tables as before, but with no constraints or triggers. We will use these two new tables to answer the last question

In [ ]:
    table = "PC2"
    table2 = "Laptop2"
    database = "Assignment 4"
    column = "speed"
    # setting variables for use in the code below
    
    conn = psycopg2.connect(
        database=database, 
        user='postgres', 
        password=Admin_Pass, 
        host='127.0.0.1', 
        port= '5434'
    )
    conn.autocommit = True
    # open connection
    cursor = conn.cursor()
    # Creating a cursor object
    
    primary_keys = get_primary_keys(table, database)
    
    SQL_Command = sql.SQL("SELECT * FROM {working_table} \n"\
                           "WHERE {col} > (SELECT AVG({col})\n"\
                                          "\t\tFROM {working_table2})\n"\
                                          "\t\tORDER BY {keys}")\
                            .format(working_table=sql.Identifier(table),\
                                    working_table2=sql.Identifier(table2),\
                                    col = sql.Identifier(column),\
                               keys = sql.SQL(', ').join(sql.Identifier(key)\
                                 for key in primary_keys))
                             # these join statements are needed
                             # in order to use the columns selected 
                             # im the SQL statement)) 
    # Establishing SQL Command  
    cursor.execute(SQL_Command, table)
    # Establishing cursor for the table picked by the user 
    column_names = [desc[0] for desc in cursor.description]
    # saving a list of the column names for the table picked by the user 
    results = cursor.fetchall()
    # gets all of the data from the table
    rowIDs = []     
    # initialize a list of the row ids
    row = 1  
    for result in results:       
        rowIDs.append(row)
        row = row+1
        # starts from 1 instead of zero for rows 
    print(tabulate(results, headers= column_names, tablefmt='psql', showindex=rowIDs))
    # displays the table
    cursor.close()
    # close the cursor
    conn.close()    
    # Closing the connection     

Here we can see all of the PC models with speeds higher than the average speed of Laptops.